# A comparison of the number of pulses in the original and decomposed graphs for large dense graphs from MQLib library.

This notebook computes the number of pulses in the original graphs and in the decomposed graphs using binary and exponential decompositions for the ten densest weighted graphs from MQLib library with fewer than 500,000 edges.

The results are stored in a CSV file `data/MQLib_large_graphs_decomposition.csv` with the following columns:
- `instance_name`: the name of the instance
- `decomposition_type`: the type of decomposition (`Original` or `Binary` or `Exponential`)
- `decomposition_epsilon`: the epsilon value used for the decomposition
- `number_of_pulses`: the number of pulses in the union-of-stars decomposition of the constituent graphs

To generate the plot for Figure 13 in the paper, use the file `plot-large-graphs.ipynb` under the `paper-plots` folder.

In [2]:
from importlib import reload
import os
import logging
import pandas as pd

logging.basicConfig(level=logging.WARNING)

os.chdir('..')


In [3]:
import src.mqlib_utils
from src.decomposition import ExponentialDecomposition, BinaryDecomposition

reload(src.mqlib_utils)

<module 'src.mqlib_utils' from '/Users/Jai/Documents/GitHub/union-of-stars-with-sparsification/src/mqlib_utils.py'>

## Helper functions to compute the number of pulses in a decomposition

In [4]:
def star_decomposition(graph):
    graph_copy = graph.copy()
    decomposition_vertices = []

    # While there are still edges in the graph, find the center of the star (the node with the highest degree)
    while graph_copy.number_of_edges() > 0:
        center = max(graph_copy.nodes, key=lambda x: graph_copy.degree(x))
        graph_copy.remove_node(center)      # Remove the center node from the graph
    
        decomposition_vertices.append(center)

    return decomposition_vertices


def decomposition_union_of_stars_pulses(constituent_graphs):
    n_pulses = 0
    for g in constituent_graphs:
        n_pulses += 3 * len(star_decomposition(g))
        
    return n_pulses + 1


# Load MQLib instance metadata and filter for large dense graphs

In [5]:
df_metadata = pd.read_csv('data/MQLib_instances_metadata.csv')          # Load metadata
df_metadata = df_metadata[df_metadata['Positive weighted'] == True]     # Pick only positive weighted graphs
df_metadata = df_metadata[df_metadata['Number of unique weights'] > 1]  # Pick only weighted graphs
df_metadata = df_metadata[df_metadata['Number of edges'] <= 500000]     # Pick only graphs with less than 500k edges     
df_metadata['Sparsity'] = df_metadata['Number of edges']/df_metadata['Number of vertices']  # Compute sparsity
df_metadata = df_metadata.sort_values(by='Sparsity', ascending=False)                       # Sort by sparsity
list_of_large_instances = df_metadata['Instance'].head(10).tolist()     # Pick the 10 densest graphs
list_of_large_instances

['g002035',
 'g001633',
 'g001737',
 'g002119',
 'g002964',
 'g000243',
 'g000562',
 'g000561',
 'g002983',
 'g001545']

# Run experiment for each of the ten large graphs

In [7]:
decomposition_types = ['binary', 'exponential']
decomposition_epsilons = [0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5]
decomposition_epsilons.reverse()

df = pd.DataFrame(columns=['instance_name', 'decomposition_type', 'decomposition_epsilon', 'number_of_pulses']
)
# Change index to 'instance_name', 'decomposition_type', 'decomposition_epsilon'
df = df.set_index(['instance_name', 'decomposition_type', 'decomposition_epsilon'])


In [11]:
for instance_name in list_of_large_instances:               # For each instance in the list of large instances
    instance = src.mqlib_utils.MQLibInstance(instance_name=instance_name, save=True)    # Load the instance from MQLib
    print('Instance name:', instance_name) 
    print(instance.graph.number_of_nodes(), instance.graph.number_of_edges())
    
    # The number of pulses for the original graph in union-of-stars is 3 * number of edges + 1 since it is a weighted graph (see https://arxiv.org/pdf/2011.08165 Theorem 5)
    # Store the number of pulses in the dataframe
    df.loc[(instance_name, 'Original', 0.0)] = 3 * instance.graph.number_of_edges() + 1
    print('\tOriginal:', 3 * instance.graph.number_of_edges() + 1)
    
    # For each epsilon, compute the decomposition and the number of pulses in the union-of-stars decomposition
    for decomposition_epsilon in decomposition_epsilons:
        print('\tDecomposition epsilon:', decomposition_epsilon)
        
        # Compute the binary decomposition
        decomp_bin = BinaryDecomposition(graph=instance.graph, epsilon=decomposition_epsilon)
        decomp_bin.compute_decomposition()
        constituent_graphs = decomp_bin.constituent_graphs
        # Compute the number of pulses in the union-of-stars decomposition
        n_pulses = decomposition_union_of_stars_pulses(constituent_graphs)
        
        print('\t\tBinary:', n_pulses)
        # Store the number of pulses in the dataframe
        df.loc[(instance_name, 'Binary', decomposition_epsilon)] = n_pulses
        
        # Compute the exponential decomposition
        decomp_exp = ExponentialDecomposition(graph=instance.graph, epsilon=decomposition_epsilon)
        decomp_exp.compute_decomposition()
        constituent_graphs = decomp_exp.constituent_graphs
        # Compute the number of pulses in the union-of-stars decomposition
        n_pulses = decomposition_union_of_stars_pulses(constituent_graphs)
        
        print('\t\tExponential:', n_pulses)
        # Store the number of pulses in the dataframe
        df.loc[(instance_name, 'Exponential', decomposition_epsilon)] = n_pulses

# Save the dataframe to a CSV file
df.to_csv('data/MQLib_large_graphs_decomposition.csv')

Instance name: g002035
991 466321
	Original: 1398964
	Decomposition epsilon: 0.5
		Binary: 58780
		Exponential: 77164
	Decomposition epsilon: 0.2
		Binary: 62770
		Exponential: 146383
	Decomposition epsilon: 0.1
		Binary: 65707
		Exponential: 233041
	Decomposition epsilon: 0.05
		Binary: 68653
		Exponential: 357127
	Decomposition epsilon: 0.02
		Binary: 72505
		Exponential: 577321
	Decomposition epsilon: 0.01
		Binary: 75442
		Exponential: 769042
	Decomposition epsilon: 0.005
		Binary: 78382
		Exponential: 955969
Instance name: g001633
955 446118
	Original: 1338355
	Decomposition epsilon: 0.5
		Binary: 29281
		Exponential: 74041
	Decomposition epsilon: 0.2
		Binary: 33058
		Exponential: 156469
	Decomposition epsilon: 0.1
		Binary: 35881
		Exponential: 265021
	Decomposition epsilon: 0.05
		Binary: 38707
		Exponential: 421147
	Decomposition epsilon: 0.02
		Binary: 42430
		Exponential: 685258
	Decomposition epsilon: 0.01
		Binary: 45265
		Exponential: 887686
	Decomposition epsilon: 0.005
